In [1]:
import stats

In [2]:
data = stats.get_data_active("TritonmanDraftLegal2025")

In [3]:
data[data.Gender == "Male"]

,Name,Gender,Age,Total,Swim,Bike,Run,T1,T2
0,Edoardo Giuseppe Leone,Male,23,00:52:02,00:05:34,00:30:27,00:15:16,00:00:22,00:00:22
1,Matheus Menezes,Male,23,00:52:03,00:05:33,00:30:31,00:15:07,00:00:25,00:00:25
2,Nathan Cohen,Male,27,00:52:36,00:05:38,00:30:23,00:15:56,00:00:18,00:00:19
3,Wiktor Rajca,Male,23,00:52:47,00:05:38,00:30:22,00:15:48,00:00:27,00:00:30
4,Nicholas Dorn,Male,25,00:52:56,00:05:37,00:30:22,00:16:02,00:00:27,00:00:26
...,...,...,...,...,...,...,...,...,...
85,Ryan Mongan,Male,59,01:06:29,00:07:23,00:37:04,00:20:54,00:00:28,00:00:36
86,Miguel Reyes,Male,21,01:07:25,00:06:38,00:38:54,00:20:10,00:00:46,00:00:55
87,Jairo Alejandro Angel Escobar,Male,34,01:07:31,00:07:09,00:38:28,00:20:36,00:00:40,00:00:35
90,Leo Tian,Male,19,01:07:43,00:06:51,00:38:33,00:20:45,00:00:54,00:00:38


In [4]:
data[data.Gender == "Female"]

,Name,Gender,Age,Total,Swim,Bike,Run,T1,T2
47,Bella Chirafisi,Female,23,00:59:45,00:06:23,00:34:28,00:17:38,00:00:37,00:00:38
48,Annie Meeder,Female,26,00:59:59,00:06:23,00:34:31,00:17:55,00:00:32,00:00:36
49,Erica Szczechowski,Female,27,01:00:06,00:06:23,00:34:30,00:17:42,00:00:35,00:00:54
54,Madeline Tapp,Female,23,01:01:37,00:06:24,00:34:32,00:19:31,00:00:33,00:00:36
56,Kasey Madsen,Female,24,01:02:02,00:06:30,00:35:42,00:18:16,00:00:40,00:00:52
58,Ava Smith,Female,23,01:02:30,00:06:46,00:35:11,00:19:17,00:00:38,00:00:36
59,Thea Zalunardo,Female,24,01:02:44,00:06:44,00:35:31,00:19:07,00:00:39,00:00:42
60,Elizabeth Harita,Female,21,01:03:10,00:06:38,00:35:40,00:19:42,00:00:31,00:00:36
62,Fiona Davoudi,Female,21,01:03:19,00:06:51,00:35:16,00:19:56,00:00:29,00:00:45
63,Siwen Cui,Female,21,01:03:24,00:06:42,00:36:27,00:18:46,00:00:45,00:00:43
